In [17]:
import numpy as np
import pandas as pd
import re
import math

In [19]:
def extract():
    file = pd.read_csv('Data_p2.csv',encoding = 'latin-1')
    print(file['v1'].value_counts(normalize = True))
    print()
    
    data = np.asarray(file)
    train_size = 0.7

    np.random.shuffle(data)
    x_train = []
    y_train = []
    x_test = []
    y_test = []

    for i in range(len(data)):
        if len(x_train) < 0.7 * len(data):
            x_train.append(data[i][1])
            y_train.append(data[i][0])
        else:
            x_test.append(data[i][1])
            y_test.append(data[i][0])
    return x_train, y_train, x_test, y_test

In [20]:
def predictorCleaning(msg):
    msg = msg.lower()
    msg = re.sub(r'[^\w\s]', ' ',msg)
    msg = msg.split(' ')
    x = []
    for word in msg:
        if len(word):
            x.append(word)
    msg = x    
    return msg


In [21]:
def dataCleaning(x_train):
    for i in range(len(x_train)):
        x_train[i] = x_train[i].lower()
        x_train[i] = re.sub(r'[^\w\s]', ' ', x_train[i])
        x_train[i] = x_train[i].split(' ')
        x = []
        for word in x_train[i]:
            if len(word):
                x.append(word)
        x_train[i] = x
    return x_train


In [22]:
def spam_predictor(msg):
    spam_prob = spam_count/len(x_train)
    ham_prob = ham_count/len(x_train) 
    words_in_msg = predictorCleaning(msg)
    for word in words_in_msg:
        if  word in spam:
            spam_prob *= spam[word]
        if  word in ham:
            ham_prob *= ham[word]
            
    if spam_prob >= ham_prob:
        return 1
    else:
        return 0 

In [12]:
x_train,y_train,x_test,y_test = extract()

print('Test Data')
print(len(x_train))
print(pd.DataFrame(y_train).value_counts(normalize = True))
print()
print('Train Data')
print(len(x_test))
print(pd.DataFrame(y_test).value_counts(normalize = True))

print()

x_train = dataCleaning(x_train)


print()

vocab = []
for row in x_train:
    for word in row:
        vocab.append(word)

vocab = list(set(vocab))
len(vocab)

print()

spam = {}
ham = {}
all_words = {}
spam_word_count = 0
ham_word_count = 0
total_words = 0
spam_count = 0
ham_count = 0

for i,msg in enumerate(x_train):
    if y_train[i] == 'spam':
        spam_count += 1
        for word in msg:
            if word in all_words:
                all_words[word] =  1
            else:
                all_words[word] = 1
            total_words += 1
            spam_word_count += 1
            if word in spam:
                spam[word] = 1
            else:
                spam[word] = 1
    elif y_train[i] == 'ham':
        ham_count += 1
        for word in msg:
            if word in all_words:
                all_words[word] =  1
            else:
                all_words[word] = 1
            total_words += 1
            ham_word_count += 1
            if word in ham:
                ham[word] = 1
            else:
                ham[word] = 1
                
print("Number of Unique words: ",len(all_words))
print("Ham Words: ")
print(len(ham))
print(ham)
print("Spam Words: ")
print(len(spam))
print(spam)

print()
alpha = 1

for word in all_words:
    if word in spam:
        prob = (1 + alpha)/(spam_count + alpha*len(all_words))
        spam[word] = prob
    else:
        spam[word] = (alpha/(spam_count + alpha*len(all_words))) 
    if word in ham:
        prob = (1 + alpha)/(ham_count + alpha*len(all_words))
        ham[word] = prob
    else:
        ham[word] = (alpha/(ham_count + alpha*len(all_words)))   

spam_pred = []
for msg in x_test:
    if spam_predictor(msg):
        spam_pred.append('spam')
    else:
        spam_pred.append('ham')

error = 0

for i in range(len(y_test)):
    if y_test[i] != spam_pred[i] :
        error += 1


print()
print('Analysis:')
error /= len(y_test)
print('Accuracy = ',(1-error)*100)
print('Error = ' ,error*100)

ham     0.865937
spam    0.134063
Name: v1, dtype: float64

Test Data
3901
ham     0.868495
spam    0.131505
dtype: float64

Train Data
1671
ham     0.859964
spam    0.140036
dtype: float64



Number of Unique words:  7176
Ham Words: 
5682
{'is': 1, 'there': 1, 'any': 1, 'training': 1, 'tomorrow': 1, 'yo': 1, 'carlos': 1, 'a': 1, 'few': 1, 'friends': 1, 'are': 1, 'already': 1, 'asking': 1, 'me': 1, 'about': 1, 'you': 1, 'working': 1, 'at': 1, 'all': 1, 'this': 1, 'weekend': 1, 'hi': 1, 'darlin': 1, 'how': 1, 'was': 1, 'work': 1, 'did': 1, 'u': 1, 'get': 1, 'into': 1, 'trouble': 1, 'ijust': 1, 'talked': 1, 'to': 1, 'your': 1, 'mum': 1, 'morning': 1, 'i': 1, 'had': 1, 'really': 1, 'good': 1, 'time': 1, 'last': 1, 'night': 1, 'im': 1, 'goin': 1, 'out': 1, 'soon': 1, 'but': 1, 'call': 1, 'if': 1, 'can': 1, 'wiskey': 1, 'brandy': 1, 'rum': 1, 'gin': 1, 'beer': 1, 'vodka': 1, 'scotch': 1, 'shampain': 1, 'wine': 1, 'kudi': 1, 'yarasu': 1, 'dhina': 1, 'vaazhthukkal': 1, 'excellent': 1, 'll': 1

In [16]:
def confusion_mat(y_test,y_pred,lamb):

    for i in range(len(y_test)):
        if(y_test[i]== 'spam' and y_pred[i]== 'spam'):
            lamb[1][1] += 1
        elif(y_test[i]== 'ham' and y_pred[i]== 'spam'):
            lamb[0][1] += 1
        elif(y_test[i]== 'ham' and y_pred[i]== 'ham'):
            lamb[0][0] += 1
        elif(y_test[i]== 'spam' and y_pred[i]== 'ham' ):
            lamb[1][0] += 1
        else:
            continue
    return lamb

lamb=np.zeros([2, 2], dtype = int)


conf_mat=confusion_mat(y_test,spam_pred,lamb)
print("Confusion Matrix: ")
print(conf_mat)

print("Analysis according to the Confusion matrix:\n")
i = 1
accuracy=(conf_mat[0][0]+conf_mat[1][1])/(conf_mat[0][0]+conf_mat[0][1]+conf_mat[1][0]+conf_mat[1][1])

print(i,"-","Accuracy:",accuracy*100)
 
i = i+1

recall=conf_mat[0][0]/(conf_mat[1][0]+conf_mat[0][0])

print(i,"-","Recall:",recall*100)
i= i+1

true_negative_rate=conf_mat[1][1]/(conf_mat[0][1]+conf_mat[1][1])
# i = i+1

print(i,"-","True Negative rate:",true_negative_rate*100)
i = i+1

precision=conf_mat[0][0]/(conf_mat[0][1]+conf_mat[0][0])

print(i,"-","Precision:",precision*100)

print()

Confusion Matrix: 
[[881 556]
 [  5 229]]
Analysis according to the Confusion matrix:

1 - Accuracy: 66.42728904847397
2 - Recall: 99.43566591422122
3 - True Negative rate: 29.17197452229299
4 - Precision: 61.30828114126653

